In [2]:
import sys
sys.path.append("../")
import re
from pathlib import Path
import os
import numpy as np
import scipy.sparse as sp
import time
import pandas as pd
from sklearn import svm
from random import sample
import sklearn.model_selection as model_selection
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import itertools
import operator
os.chdir('../')

In [29]:
trojans = ["BankBot", "Boxer", "FakeInst", "FakeAV", "FakeDoc", "Gorpo", "Kemoge", 
                      "Ksapp", "MobileTX", "Mtk", "Nandrobox", "RuMMS", "SmsZombie", "SpyBubble", "Stealer", 
                      "Steek", "Vidro", "VikingHorde", "Vmvol"]

In [19]:
adware = ["Andup", "Kyview", "Youmi", "Minimob"]

In [14]:
backdoor = ["DroidKungFu", "FakeAngry", "Fjcon", "GingerMaster", "Obad", "Univert"]

In [2]:
ransom = ["Koler", "Roop", "SimpleLocker"]

In [30]:
mal_oc = {}
pathname = "data/apk_data/"
for i in trojans:
    listOfApps = os.listdir(pathname + i)
    for j in listOfApps:
        if not os.path.exists(pathname + i + "/" + j + "/" + "code_blocks.txt"):
            continue
        with open(pathname + i + "/" + j + "/" + "code_blocks.txt", 'r') as f:
            for line in f:
                lst = line.split(",")
                while len(lst) > 1:
                    k = lst.pop()
                    k = k.strip()
                    k = k.replace('"', '')
                    ind = k.index(")")
                    k = k[:ind+2]
                    if "[" in k:
                        k.replace("[", '')
                    for l in lst:
                        l = l.strip()
                        l = l.replace('"', '')
                        indl = l.index(")")
                        l = l[:indl+2]
                        if "[" in l:
                            l.replace("[", '')
                        if k + ", " + l not in mal_oc:
                            if l + ", " + k not in mal_oc:
                                mal_oc[k + ", " + l] = [1]
                            else:
                                mal_oc[l + ", " + k][0] += 1
                        else:
                            mal_oc[k + ", " + l][0] += 1
so_oc = {k: v for k,v in sorted(mal_oc.items(), key=operator.itemgetter(1), reverse = True)}
df = pd.DataFrame(so_oc).transpose().rename(columns = {0: 'occurrences'}).reset_index(drop = False).rename(columns = {"index": "pairs"})

In [31]:
df["first"] = df["pairs"].apply(lambda x: x.split(",")[0])
df["second"] = df["pairs"].apply(lambda x: x.split(",")[1])
df = df[["first", "second", "occurrences"]]
df.head(10)

,first,second,occurrences
0,'Lgnu/mapping/SimpleSymbol;->readResolve()L,'Lgnu/lists/PairWithPosition;->make()L,3128052
1,'Lgnu/lists/PairWithPosition;->make()L,'Lgnu/lists/PairWithPosition;->make()L,2850168
2,'Ljava/lang/StringBuilder;->append()L,'Ljava/lang/StringBuilder;->append()L,2465470
3,'Ljava/lang/StringBuilder;->toString()L,'Ljava/lang/StringBuilder;->append()L,1476119
4,'Lkawa/standard/Scheme;->defProcStFld()V,'Lkawa/standard/Scheme;->defProcStFld()V,1362312
5,'Lcom/irnmtim/nitolob/q;->a()L,'Lcom/irnmtim/nitolob/q;->a()[,1279560
6,'Lgnu/mapping/SimpleSymbol;->readResolve()L,'Lgnu/mapping/SimpleSymbol;->readResolve()L,1215864
7,'Lgnu/lists/PairWithPosition;->make()L,'Lgnu/lists/Pair;->make()L,1047312
8,'Ljava/util/Map;->put()L,'Ljava/lang/Character;->valueOf()L,804973
9,'Ljava/io/PrintWriter;->print()V,'Ljava/io/PrintWriter;->print()V,763722


In [ ]:
beg = time.time()
co_oc = {}
pathname = "data/apk_data/"
for i in ["entertainment", "productivity"]:
    listOfApps = os.listdir(pathname + i)
    for j in listOfApps:
        with open(pathname + i + "/" + j + "/" + "code_blocks.txt", 'r') as f:
            for line in f:
                lst = line.split(",")
                while len(lst) > 1:
                    k = lst.pop()
                    k = k.strip()
                    k = k.replace('"', '')
                    ind = k.index(")")
                    k = k[:ind+2]
                    if not k[-1].isalpha():
                        k = k[:-1]
                    if "[" in k:
                        k.replace("[", '')
                    for l in lst:
                        l = l.strip()
                        l = l.replace('"', '')
                        indl = l.index(")")
                        l = l[:indl+2]
                        if not l[-1].isalpha():
                            l = l[:-1]
                        if "[" in l:
                            l.replace("[", '')
                        if k + ", " + l not in co_oc:
                            if l + ", " + k not in co_oc:
                                co_oc[k + ", " + l] = [1]
                            else:
                                co_oc[l + ", " + k][0] += 1
                        else:
                            co_oc[k + ", " + l][0] += 1
print(time.time() - beg)
so_oc = {k: v for k,v in sorted(co_oc.items(), key=operator.itemgetter(1), reverse = True)}
benigns = pd.DataFrame(so_oc).transpose().rename(columns = {0: 'occurrences'}).reset_index(drop = False).rename(columns = {"index": "pairs"})
benigns.to_csv('SORTED_CO_OC_EP1.csv')

In [23]:
benigns = pd.read_csv("SORTED_CO_OC_EP1.csv")
benigns["first"] = benigns["pairs"].apply(lambda x: x.split(",")[0])
benigns["second"] = benigns["pairs"].apply(lambda x: x.split(",")[1])
benigns = benigns[["first", "second", "occurrences"]]
benigns.head(10)

,first,second,occurrences
0,'Landroid/content/ContentValues;->put()V,'Landroid/content/ContentValues;->put()V,78935984
1,'Lgnu/lists/PairWithPosition;->make()L,'Lgnu/lists/PairWithPosition;->make()L,35698312
2,'Ljava/lang/Integer;->valueOf()L,'Ljava/lang/Integer;->valueOf()L,23643370
3,'Lgnu/lists/PairWithPosition;->make()L,'Lgnu/mapping/SimpleSymbol;->readResolve()L,12007847
4,'Ljava/lang/StringBuilder;->append()L,'Ljava/lang/StringBuilder;->append()L,11893187
5,'Ljava/util/Map;->put()L,'Ljava/util/Map;->put()L,8838822
6,'Lfr/pcsoft/wdjava/core/ressources/messages/fr/WDResFct_fr;->a()L,'Lfr/pcsoft/wdjava/core/ressources/messages/fr/WDResFct_fr;->a()[,7770156
7,'Ljava/lang/StringBuilder;->toString()L,'Ljava/lang/StringBuilder;->append()L,5514757
8,'Lfr/pcsoft/wdjava/core/ressources/messages/fr/WDResFct_fr;->a()L,'Lfr/pcsoft/wdjava/core/ressources/messages/fr/WDResFct_fr;->a()L,3885078
9,'Lfr/pcsoft/wdjava/core/ressources/messages/fr/WDResFct_fr;->a()[,'Lfr/pcsoft/wdjava/core/ressources/messages/fr/WDResFct_fr;->a()[,3882291


In [12]:
pd.set_option('display.max_colwidth', -1)